In [1]:
import hid
import struct
import time
from pynput.mouse import Button, Controller

# --- 控制器解析部分保持不变 ---
class XboxController:
    # [最终修正] 根据你的精确报文数据得出的按钮映射
    BUTTON_MAP = {
        4: "A",
        5: "B",
        6: "X",
        7: "Y",
    }
    BUTTON_MAP_2 = {
        4: "LB",
        5: "RB",
        6: "LS", # Left Stick Click
        7: "RS", # Right Stick Click
    }

    def __init__(self, vendor_id=0x045E, product_id=0x0B12):
        self.device = None
        try:
            self.device = hid.device()
            self.device.open(vendor_id, product_id)
            self.device.set_nonblocking(True)
            print("Connected:", self.device.get_manufacturer_string(),
                  self.device.get_product_string())
        except OSError as e:
            print(f"Error opening device: {e}")
            self.device = None

    def close(self):
        if self.device:
            self.device.close()

    def read(self):
        if not self.device:
            return None
        
        data = self.device.read(64, timeout_ms=1) 
        if not data or len(data) < 18:
            return None

        raw = bytes(data)
        
        buttons1_raw = raw[4]
        buttons2_raw = raw[5]

        lt, rt = struct.unpack_from("<HH", raw, 6)
        lx, ly, rx, ry = struct.unpack_from("<hhhh", raw, 10)

        buttons = self._decode_buttons(buttons1_raw, self.BUTTON_MAP)
        buttons.update(self._decode_buttons(buttons2_raw, self.BUTTON_MAP_2))

        # 返回一个统一的状态字典，便于后续处理
        return {
            "buttons": buttons,
            "lt": lt / 1023.0,
            "rt": rt / 1023.0,
            "lx": self._normalize_axis(lx),
            "ly": self._normalize_axis(ly),
            "rx": self._normalize_axis(rx),
            "ry": self._normalize_axis(ry),
        }

    def _decode_buttons(self, bitmask, button_map):
        return {name: bool(bitmask & (1 << bit)) for bit, name in button_map.items()}

    def _normalize_axis(self, v):
        v = int(v)
        if v < 0: return max(-1.0, v / 32768.0)
        else: return min(1.0, v / 32767.0)

# ==============================================================================
# ======================== 新的配置和动作处理系统 ==========================
# ==============================================================================

# 基础动作类 (所有其他动作类的父类)
class Action:
    """所有动作的基类，定义了 update 接口。"""
    def update(self, state, last_state, mouse):
        # state: 当前控制器状态
        # last_state: 上一帧的控制器状态
        # mouse: pynput 的 mouse 控制器实例
        pass

# 动作类：鼠标移动
class MouseMoveAction(Action):
    """处理摇杆到鼠标移动的映射。"""
    def __init__(self, x_axis, y_axis, sensitivity, deadzone):
        self.x_axis = x_axis
        self.y_axis = y_axis
        self.sensitivity = sensitivity
        self.deadzone = deadzone

    def update(self, state, last_state, mouse):
        lx = state[self.x_axis]
        ly = state[self.y_axis]
        
        # 应用死区
        if abs(lx) < self.deadzone: lx = 0
        if abs(ly) < self.deadzone: ly = 0
        
        if lx != 0 or ly != 0:
            # 使用立方曲线使低速移动更精确
            x_move = (lx ** 3) * self.sensitivity
            y_move = -(ly ** 3) * self.sensitivity
            mouse.move(x_move, y_move)

# 动作类：鼠标点击
class ClickAction(Action):
    """处理按钮到鼠标点击的映射。"""
    def __init__(self, controller_button, mouse_button):
        self.controller_button = controller_button
        self.mouse_button = mouse_button

    def update(self, state, last_state, mouse):
        is_pressed = state['buttons'].get(self.controller_button, False)
        was_pressed = last_state['buttons'].get(self.controller_button, False) if last_state else False

        if is_pressed and not was_pressed:
            mouse.press(self.mouse_button)
        elif not is_pressed and was_pressed:
            mouse.release(self.mouse_button)

# 动作类：鼠标滚动（带延迟和重复）
class ScrollAction(Action):
    """处理按钮到平滑滚动的映射，自带状态管理。"""
    def __init__(self, controller_button, scroll_speed, initial_delay, repeat_rate):
        self.controller_button = controller_button
        self.scroll_speed = scroll_speed
        self.initial_delay = initial_delay
        self.repeat_rate = repeat_rate
        
        # 内部状态变量，每个实例独立
        self.pressed = False
        self.next_scroll_time = 0

    def update(self, state, last_state, mouse):
        is_down = state['buttons'].get(self.controller_button, False)
        current_time = time.time()

        if is_down:
            if not self.pressed: # 第一次按下
                mouse.scroll(0, self.scroll_speed)
                self.pressed = True
                self.next_scroll_time = current_time + self.initial_delay
            elif current_time >= self.next_scroll_time: # 持续按住
                mouse.scroll(0, self.scroll_speed)
                self.next_scroll_time = current_time + self.repeat_rate
        else: # 松开按键
            self.pressed = False


if __name__ == "__main__":
    # --------------------------------------------------------------------------
    # --- ✨✨✨ 新的配置区域 ✨✨✨ ---
    # 在这里定义你的控制器映射，而不是写一堆if语句。
    # 想添加新功能？只需创建一个新的Action类并在这里添加实例。
    # --------------------------------------------------------------------------
    ACTION_CONFIG = [
        # 鼠标移动：将左摇杆(lx, ly)映射到鼠标移动
        MouseMoveAction(
            x_axis='lx',
            y_axis='ly',
            sensitivity=25,
            deadzone=0.15
        ),
        
        # 鼠标点击：将A键映射到鼠标左键
        ClickAction(
            controller_button='A', 
            mouse_button=Button.left
        ),

        # 鼠标点击：将B键映射到鼠标右键
        ClickAction(
            controller_button='B', 
            mouse_button=Button.right
        ),

        # 鼠标滚动：将RB键映射到向下滚动
        ScrollAction(
            controller_button='RB',
            scroll_speed=-15, # 负值通常是向下
            initial_delay=0.3,
            repeat_rate=0.05
        ),

        # 鼠标滚动：将LB键映射到向上滚动
        ScrollAction(
            controller_button='LB',
            scroll_speed=15, # 正值是向上
            initial_delay=0.3,
            repeat_rate=0.05
        ),
    ]

    try:
        xbox = XboxController()
        if not xbox.device:
            raise OSError("Controller not found or could not be opened.")
        
        mouse = Controller()

        print("\nController mapped successfully! Mouse control is active.")
        print("Configuration loaded. Ctrl+C to exit.")
        print("-" * 50)
        
        last_state = None
        last_print_time = 0

        # --------------------------------------------------------------------------
        # --- ✨ 主循环现在变得非常简洁 ✨ ---
        # 它只负责读取状态并将其分发给配置好的动作处理器。
        # --------------------------------------------------------------------------
        while True:
            state = xbox.read()
            if state:
                # 遍历配置中的每一个动作，并更新它们的状态
                for action in ACTION_CONFIG:
                    action.update(state, last_state, mouse)
                
                # 更新上一帧的状态
                last_state = state

                # 调试输出 (保持不变)
                current_time = time.time()
                if current_time - last_print_time > 0.1:
                    pressed_buttons = sorted([name for name, pressed in state["buttons"].items() if pressed])
                    print(
                        f"Stick:({state['lx']:.2f}, {state['ly']:.2f}) "
                        f"LT:{state['lt']:.2f} RT:{state['rt']:.2f} "
                        f"Buttons: {pressed_buttons}      ", end='\r'
                    )
                    last_print_time = current_time
            else:
                # 如果没有读到数据，短暂休眠避免CPU空转
                time.sleep(0.001)

    except OSError as e:
        print(f"\nError: {e}")
    except KeyboardInterrupt:
        print("\nExiting.")
    finally:
        if 'xbox' in locals() and xbox:
            xbox.close()

Connected: Microsoft Controller

Controller mapped successfully! Mouse control is active.
Configuration loaded. Ctrl+C to exit.
--------------------------------------------------
Stick:(-0.03, 0.03) LT:0.00 RT:0.00 Buttons: []           
Exiting.
